In [ ]:
import os, cv2, random
import matplotlib.pyplot as plt
from aaa_image_enhancement.image_defects_detection import detect_image_defects
from aaa_image_enhancement.image_features import ImageParams


# folder_path = "../imgs"
folder_path = "../craigslist_images"


def detect_blurry_images(folder_path):
    blurry_images = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            defects = detect_image_defects(cv2.imread(image_path), blur_threshold=100)

            if defects.blur:
                blurry_images.append(filename)

    return blurry_images


blurry_images = detect_blurry_images(folder_path)
print("Blurry images:")
for filename in blurry_images:
    print(filename)

# Get a list of all image files in the folder
image_files = [
    file
    for file in os.listdir(folder_path)
    if file.endswith(".jpg") or file.endswith(".jpeg")
]

# Randomly select 9 images (or fewer if not enough images are available)
num_images = min(9, len(blurry_images))
random_images = random.sample(blurry_images, num_images)

# Create a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(10, 10))

# Iterate over the selected images and plot them in the grid
for i, ax in enumerate(axes.flat):
    if i < num_images:
        img_path = os.path.join(folder_path, random_images[i])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
        ax.imshow(img)
        ax.set_title(random_images[i])
    ax.axis("off")  # Remove axis labels and ticks

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
from aaa_image_enhancement.exposure_enhancement import enhance_image_exposure

In [ ]:
# import os, cv2, random
# import matplotlib.pyplot as plt
# from aaa_image_enhancement.image_defects_detection import is_blurry
# from aaa_image_enhancement.image_features import ImageParams


# # folder_path = "../imgs"
# folder_path = "../craigslist_images"


# def detect_blurry_images(folder_path):
#     blurry_images = []

#     for filename in os.listdir(folder_path):
#         if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#             image_path = os.path.join(folder_path, filename)
#             map, score, is_blurry = is_blurry(cv2.imread(image_path), 100)

#             if is_blurry:
#                 blurry_images.append(filename)

#     return blurry_images


# blurry_images = detect_blurry_images(folder_path)
# print("Blurry images:")
# for filename in blurry_images:
#     print(filename)

# # Get a list of all image files in the folder
# image_files = [
#     file
#     for file in os.listdir(folder_path)
#     if file.endswith(".jpg") or file.endswith(".jpeg")
# ]

# # Randomly select 9 images (or fewer if not enough images are available)
# num_images = min(9, len(blurry_images))
# random_images = random.sample(blurry_images, num_images)

# # Create a 3x3 grid of subplots
# fig, axes = plt.subplots(3, 3, figsize=(10, 10))

# # Iterate over the selected images and plot them in the grid
# for i, ax in enumerate(axes.flat):
#     if i < num_images:
#         img_path = os.path.join(folder_path, random_images[i])
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
#         ax.imshow(img)
#         ax.set_title(random_images[i])
#     ax.axis("off")  # Remove axis labels and ticks

# plt.tight_layout()  # Adjust spacing between subplots
# plt.show()

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt


def estimate_blur(image, threshold=100):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur_map = cv2.Laplacian(gray, cv2.CV_64F).var()
    score = int(blur_map)
    return blur_map, score, blur_map < threshold


def detect_blurry_images(folder_path):
    blurry_images = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            defects = detect_image_defects(cv2.imread(image_path), blur_threshold=150)

            if defects.blur:
                blurry_images.append(filename)

    return blurry_images


def deblur_image(image):
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened_image = cv2.filter2D(image, -1, sharpen_kernel)
    return sharpened_image


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Detect blurry images
blurry_images = detect_blurry_images(folder_path)

# Randomly select N blurry images
N = 5
random_blurry_images = random.sample(blurry_images, N)

# Create a figure with two columns
fig, axes = plt.subplots(N, 2, figsize=(10, 15))

# Iterate over the selected blurry images
for i, image_file in enumerate(random_blurry_images):
    image_path = os.path.join(folder_path, image_file)

    # Read the input image
    image = cv2.imread(image_path)

    # Deblur the image
    deblurred_image = deblur_image(image)

    # Convert images from BGR to RGB for Matplotlib display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    deblurred_image_rgb = cv2.cvtColor(deblurred_image, cv2.COLOR_BGR2RGB)

    # Display the original and deblurred images side by side
    axes[i, 0].imshow(image_rgb)
    axes[i, 0].set_title("Original Blurry Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(deblurred_image_rgb)
    axes[i, 1].set_title("Deblurred Image")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def estimate_haziness(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the dark channel prior
    dark_channel = cv2.erode(gray, np.ones((15, 15), np.uint8))

    # Estimate the atmospheric light
    atmospheric_light = np.max(dark_channel)

    # Compute the transmission map
    transmission_map = 1 - (dark_channel / atmospheric_light)

    gray = gray.astype(np.float32)
    transmission_map = transmission_map.astype(np.float32)

    # Refine the transmission map using guided filter
    transmission_map_refined = cv2.ximgproc.guidedFilter(
        gray, transmission_map, 50, 1e-3
    )
    # Estimate the haziness score
    haziness_score = 1 - np.mean(transmission_map_refined)

    return haziness_score, transmission_map_refined


def detect_hazy_images(folder_path, threshold=0.6):
    hazy_images = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            haziness_score, _ = estimate_haziness(image)

            if haziness_score > threshold:
                hazy_images.append(filename)

    return hazy_images


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Detect hazy images and print their filenames
hazy_images = detect_hazy_images(folder_path)
print("Hazy images:")
for filename in hazy_images:
    print(filename)

# Display a 3x3 grid of random hazy images
num_images = min(9, len(hazy_images))
random_hazy_images = np.random.choice(hazy_images, num_images, replace=False)

fig, axes = plt.subplots(3, 3, figsize=(10, 10))

for i, ax in enumerate(axes.flat):
    if i < num_images:
        img_path = os.path.join(folder_path, random_hazy_images[i])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.set_title(random_hazy_images[i])
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import image_dehazer

folder_path = "../craigslist_images"

# Get a list of all image files in the folder
image_files = [
    file
    for file in os.listdir(folder_path)
    if file.endswith(".jpg") or file.endswith(".jpeg")
]

# Randomly select 5 images
num_images = 5
random_images = random.sample(image_files, num_images)

# Create a figure with two columns
fig, axes = plt.subplots(num_images, 2, figsize=(10, 15))

# Iterate over the selected images
for i, image_file in enumerate(random_images):
    image_path = os.path.join(folder_path, image_file)

    # Read the input image
    HazeImg = cv2.imread(image_path)

    # Remove haze from the image
    HazeCorrectedImg, HazeTransmissionMap = image_dehazer.remove_haze(
        HazeImg, showHazeTransmissionMap=False, C0=50, C1=500
    )

    # Convert images from BGR to RGB for Matplotlib display
    HazeImg_RGB = cv2.cvtColor(HazeImg, cv2.COLOR_BGR2RGB)
    HazeCorrectedImg_RGB = cv2.cvtColor(HazeCorrectedImg, cv2.COLOR_BGR2RGB)

    # Display the original and dehazed images side by side
    axes[i, 0].imshow(HazeImg_RGB)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(HazeCorrectedImg_RGB)
    axes[i, 1].set_title("Dehazed Image")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import cv2
import numpy as np
from skimage.restoration import estimate_sigma


def is_noisy(image, threshold=2.0):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Estimate the standard deviation of the noise
    sigma = estimate_sigma(gray, average_sigmas=True)

    # Check if the estimated noise level exceeds the threshold
    return sigma > threshold


def detect_noisy_images(folder_path, threshold=2.0):
    noisy_images = []

    # Iterate over all image files in the folder
    for filename in os.listdir(folder_path):
        if (
            filename.endswith(".jpg")
            or filename.endswith(".jpeg")
            or filename.endswith(".png")
        ):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            if is_noisy(image, threshold):
                noisy_images.append(filename)

    return noisy_images


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Detect noisy images and print their filenames
noisy_images = detect_noisy_images(folder_path)
print("Noisy images:")
for filename in noisy_images:
    print(filename)

num_images = 5
random_images = random.sample(noisy_images, num_images)

# Create a figure with two columns
fig, axes = plt.subplots(num_images, 2, figsize=(10, 15))

# Iterate over the selected images
for i, image_file in enumerate(random_images):
    image_path = os.path.join(folder_path, image_file)

    # Read the input image
    HazeImg = cv2.imread(image_path)

    # Remove haze from the image
    HazeCorrectedImg, HazeTransmissionMap = image_dehazer.remove_haze(
        HazeImg, showHazeTransmissionMap=False, C0=50, C1=500
    )

    # Convert images from BGR to RGB for Matplotlib display
    HazeImg_RGB = cv2.cvtColor(HazeImg, cv2.COLOR_BGR2RGB)
    HazeCorrectedImg_RGB = cv2.cvtColor(HazeCorrectedImg, cv2.COLOR_BGR2RGB)

    # Display the original and dehazed images side by side
    axes[i, 0].imshow(HazeImg_RGB)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(HazeCorrectedImg_RGB)
    axes[i, 1].set_title("Dehazed Image")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import random
import cv2
import numpy as np
from skimage.exposure import is_low_contrast as ski_is_low_contrast
import matplotlib.pyplot as plt


def detect_low_contrast(image, threshold=0.35):
    return ski_is_low_contrast(image, fraction_threshold=threshold)


def detect_poor_white_balance(image):
    b, g, r = cv2.split(image)
    avg_b = np.mean(b)
    avg_g = np.mean(g)
    avg_r = np.mean(r)

    if abs(avg_b - avg_g) > 20 or abs(avg_b - avg_r) > 20 or abs(avg_g - avg_r) > 20:
        return True
    return False


def enhance_image(image):
    # Apply white balancing
    wb = cv2.xphoto.createSimpleWB()
    wb.setP(0.2)
    white_balanced_image = wb.balanceWhite(image)

    # Apply contrast enhancement
    lab = cv2.cvtColor(white_balanced_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1, tileGridSize=(8, 8))
    enhanced_l = clahe.apply(l)
    enhanced_lab = cv2.merge((enhanced_l, a, b))
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    return enhanced_image


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Get a list of all image files in the folder
image_files = [
    file
    for file in os.listdir(folder_path)
    if file.endswith(".jpg") or file.endswith(".jpeg")
]

# Randomly select 5 images
num_images = 5
random_images = random.sample(image_files, num_images)

# Create a figure with two columns
fig, axes = plt.subplots(num_images, 2, figsize=(15, 15))

# Iterate over the selected images
for i, image_file in enumerate(random_images):
    image_path = os.path.join(folder_path, image_file)

    # Read the input image
    image = cv2.imread(image_path)

    # Detect low contrast and poor white balance
    is_low_contrast = detect_low_contrast(image)
    is_poor_white_balance = detect_poor_white_balance(image)

    if is_low_contrast or is_poor_white_balance:
        # Enhance the image
        enhanced_image = enhance_image(image)
    else:
        enhanced_image = image

    # Convert images from BGR to RGB for Matplotlib display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    enhanced_image_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)

    # Display the original and enhanced images side by side
    axes[i, 0].imshow(image_rgb)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(enhanced_image_rgb)
    axes[i, 1].set_title("Enhanced Image")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt


def is_low_light(image, threshold=80):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate average pixel intensity
    avg_intensity = np.mean(gray)
    # Check if average intensity is below the threshold
    return avg_intensity < threshold


def enhance_low_light(image):
    # Convert image to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels
    y, cr, cb = cv2.split(ycrcb)

    # Apply histogram equalization to the Y channel
    equalized_y = cv2.equalizeHist(y)

    # Merge the equalized Y channel with the original Cr and Cb channels
    equalized_ycrcb = cv2.merge((equalized_y, cr, cb))

    # Convert the equalized YCrCb image back to BGR
    equalized_bgr = cv2.cvtColor(equalized_ycrcb, cv2.COLOR_YCrCb2BGR)

    # Apply bilateral filter for noise reduction
    denoised = cv2.bilateralFilter(equalized_bgr, 9, 75, 75)

    # Apply unsharp masking for sharpening
    blurred = cv2.GaussianBlur(denoised, (0, 0), 3)
    sharpened = cv2.addWeighted(denoised, 1.5, blurred, -0.5, 0)

    return sharpened


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Get a list of all image files in the folder
image_files = [
    file
    for file in os.listdir(folder_path)
    if file.endswith(".jpg") or file.endswith(".jpeg")
]

# Randomly select images until we have 5 low light images or all images are checked
num_images = 5
low_light_images = []
random.shuffle(image_files)

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    if is_low_light(image):
        low_light_images.append((image, image_file))
        if len(low_light_images) == num_images:
            break

# Create a figure with two columns
num_low_light_images = len(low_light_images)
fig, axes = plt.subplots(
    num_low_light_images, 2, figsize=(10, 5 * num_low_light_images)
)

# Iterate over the low light images
for i, (image, image_file) in enumerate(low_light_images):
    # Enhance the low light image
    enhanced_image = enhance_image_exposure(image, gamma=0.6, lambda_=0.15)

    # Convert images from BGR to RGB for Matplotlib display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    enhanced_image_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)

    # Display the original and enhanced images side by side
    axes[i, 0].imshow(image_rgb)
    axes[i, 0].set_title(f"Original Image - {image_file}")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(enhanced_image_rgb)
    axes[i, 1].set_title(f"Enhanced Image - {image_file}")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
from aaa_image_enhancement.image_defects_detection import detect_image_defects


def find_defective_images(folder_path, defect_type):
    defective_images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            defects = detect_image_defects(image)
            if getattr(defects, defect_type):
                defective_images.append(filename)
    return defective_images


# Specify the folder path containing the images
folder_path = "../craigslist_images"

# Specify the defect type you want to find (e.g., "blur", "low_light", "low_contrast", "poor_white_balance", "noisy")
defect_type = "low_light"

# Find images with the specified defect
defective_images = find_defective_images(folder_path, defect_type)

# Randomly select 9 defective images (or fewer if not enough images are available)
num_images = min(9, len(defective_images))
random_defective_images = random.sample(defective_images, num_images)

# Create a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

# Iterate over the selected defective images and plot them in the grid
for i, ax in enumerate(axes.flat):
    if i < num_images:
        img_path = os.path.join(folder_path, random_defective_images[i])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.set_title(random_defective_images[i])
    ax.axis("off")

plt.tight_layout()
plt.show()